In [15]:
# Data taken from https://jochen-hoenicke.de/queue/#BTC,all,count

import json 
import os.path
import math
import matplotlib.pyplot as plt
import numpy as np
from types import SimpleNamespace

PROBLEMATIC_INTERVALS = [[1516728783, 1516729440], [1515943500, 1515944160]] # problematic intervals that should not be considerated (probably the BTC nodes of the owner went offline for a while)

fee_ranges = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 17, 20, 25, 30, 40, 50, 60, 70, 80, 100, 120, 140, 170, 200, 250, 300, 400, 500, 600, 700, 800, 1000, 1200, 1400, 1700, 2000, 2500, 3000, 4000, 5000, 6000, 7000, 8000, 10000 ]

def getAverageFeeIndex(tx_count_per_fee_level):
    total_tx_count = sum(tx_count_per_fee_level)

    acc = 0
    i = 0

    # Computing the average fee
    for c in tx_count_per_fee_level:
        acc = acc + (fee_ranges[i] * c)
        i = i + 1

    avg_fee = acc/total_tx_count

    average_index = 0

    i = 1

    # Computing the index of the average fee
    while i < len(fee_ranges):
        if fee_ranges[i-1] <= avg_fee and avg_fee <= fee_ranges[i]:
            average_index = i
            break
        i = i + 1
    
    return average_index

def getTxsWithHigherFeeLevel(fee_index, tx_count_per_fee_level, tx_to_be_added):
    tx_with_higher_fee = 0 

    index = fee_index + 1

    while index < len(tx_count_per_fee_level):
        tx_with_higher_fee = tx_with_higher_fee + tx_count_per_fee_level[index] + tx_to_be_added[index]
        index = index + 1

    return tx_with_higher_fee

def confirmTransactions(fee_index, initial_tx_with_same_fee, tx_count_per_fee_level, estimated_num_tx_in_block):
    tx_with_higher_fee = 0 

    index = fee_index + 1

    while index < len(tx_count_per_fee_level):
        tx_with_higher_fee = tx_with_higher_fee + tx_count_per_fee_level[index]
        index = index + 1

    if(tx_with_higher_fee <= estimated_num_tx_in_block): # if the new block can contain transactions with the level of fee we are considering for closing channel transactions

        # 2 ways: random ordering of all the tx with this fee level or ordered queue based on arrival time

        number_of_confirmed_tx = estimated_num_tx_in_block - tx_with_higher_fee # ... of the same fee rate of the closing channel transactions

        if(initial_tx_with_same_fee >= number_of_confirmed_tx):

            initial_tx_with_same_fee -= number_of_confirmed_tx
            return initial_tx_with_same_fee, 0    

        else:
            if(initial_tx_with_same_fee > 0):
                number_of_confirmed_tx -= initial_tx_with_same_fee
                initial_tx_with_same_fee = 0
            return initial_tx_with_same_fee, number_of_confirmed_tx
    else:
        return initial_tx_with_same_fee, 0  

def isInProblematicInterval(timestamp, PROBLEMATIC_INTERVALS):
    for interval in PROBLEMATIC_INTERVALS:
        if(timestamp >= interval[0] and timestamp < interval[1]):
            return True
    return False
        
# if highest_priority == True, then txs are put in the first positions of the queue (used when we are considering a fee level that is greater than the maximum fee level in our dataset: in this case, we
# want that our transactions are those with the highest priority in the mempool)

# There will be num_zombie_channels * 2 transactions  (both from the attacker and from the victims to dispute )
def simulate(mempool_data, fee_index_attacker, num_channels, highest_priority=False, dynamic=False, alpha=0.5, beta=1, step=1):
    
    tx_to_be_added = [0] * len(fee_ranges) # when we confirm some of "our" transactions that were not actually in the mempool, we have to consider that in our dataset that same number of transactions exit the mempool, but in our simulation we have to consider as they are still in the mempool
    ln_txs_to_add_per_fee_level = []

    LN_DELAY = 1000 # in blocks

    remaining_attacker_txs = num_channels
        
    num_confirmed_victim_transactions = 0
    victim_index = 0
    victim_transactions = []  # Array of SimpleNamespace (JavaScript-like objects)

    for i in range(num_channels):
        transaction = SimpleNamespace()
        transaction.submitted = False
        transaction.confirmed = False
        transaction.confirmedBlockNumber = -1
        transaction.attackerTxConfirmedBlockNumber = -1
        transaction.fee_index = -1
        transaction.tx_with_same_fee = -1 # Transactions already present in the mempool with the same fee level
        transaction.isAttacker = False
        transaction.id = i
        victim_transactions.append(transaction)  

          
    victim_fee_indexes_cache = {}
    blocksCounter = 0
    last_total_tx_count = None
    last_tx_count_per_fee_level = None
    
    # if not highest_priority:
    #     initial_tx_with_same_fee = mempool_data[0][1][fee_index_in_ranges]

    tx_to_be_added[fee_index_attacker] = num_channels

    initial_tx_with_same_fee_attacker = mempool_data[0][1][fee_index_attacker]

    for snapshot in mempool_data:
        timestamp = snapshot[0]
        tx_count_per_fee_level = snapshot[1]
        
        total_tx_count = sum(tx_count_per_fee_level)

        if last_total_tx_count is None and last_tx_count_per_fee_level is None:
            last_total_tx_count = total_tx_count
            last_tx_count_per_fee_level = tx_count_per_fee_level
            continue # to the next loop iteration
        
        is_in_problematic_interval = isInProblematicInterval(timestamp, PROBLEMATIC_INTERVALS)

        if total_tx_count < last_total_tx_count and not is_in_problematic_interval:
            
            # New Block
            blocksCounter += 1
            
            estimated_num_tx_in_block = last_total_tx_count - total_tx_count

            # TODO: should we consider that each block contains the max number of transactions or historical data, to decide how many closing channel transactions
            # are included in this block?

            sortByFeeRateAndMempoolPosition = []

            if(victim_index > 0):
                # Then we have submitted dispute transactions from victims
                sortByFeeRateAndMempoolPosition = victim_transactions[:victim_index]
            
            if(remaining_attacker_txs > 0):
                attackerTx = SimpleNamespace()
                attackerTx.fee_index = fee_index_attacker
                attackerTx.tx_with_same_fee = 0 # ... because at least one of the attacker tx has been confirmed
                attackerTx.isAttacker = True 
                sortByFeeRateAndMempoolPosition.append(attackerTx)

            sortByFeeRateAndMempoolPosition.sort(key=lambda x: (x.fee_index, -x.tx_with_same_fee), reverse=True)

            # tx_count_per_fee_level_difference = list(np.subtract(np.array(last_tx_count_per_fee_level), np.array(tx_count_per_fee_level)))
            
            # for i in range(len(tx_count_per_fee_level_difference)):
            #     if(tx_count_per_fee_level_difference[i] < 0):
            #         tx_count_per_fee_level_difference[i] = 0

            tx_with_higher_fee = getTxsWithHigherFeeLevel(sortByFeeRateAndMempoolPosition[0].fee_index, last_tx_count_per_fee_level, tx_to_be_added) 

            number_of_tx_to_be_confirmed = estimated_num_tx_in_block - tx_with_higher_fee

            if(number_of_tx_to_be_confirmed > 0):
                # Then some of "our" transactions could pass
                # this does NOT mean that `number_of_tx_be_confirmed` of our txs will pass, it depends on the position of them in the mempool in the same fee level

                ln_txs_to_add_per_fee_level = [0] * len(fee_ranges)

                # for i in range(sortByFeeRateAndMempoolPosition[0].fee_index, len(tx_count_per_fee_level_difference)):
                #     tx_count_per_fee_level_difference[i] = 0

                for tx in sortByFeeRateAndMempoolPosition:
                    
                    if(number_of_tx_to_be_confirmed <= 0):
                        break

                    if not tx.isAttacker:
                        
                        if(tx.confirmed):
                            continue 

                        delta_txs = 0 

                        if tx.tx_with_same_fee >= number_of_tx_to_be_confirmed:
                            # also need to remove to all other txs with the same fee level, that were already in the mempool at the time of insertion
                            delta_txs = number_of_tx_to_be_confirmed
                            number_of_tx_to_be_confirmed = 0
                        else:
                            number_of_tx_to_be_confirmed -= tx.tx_with_same_fee
                            delta_txs = tx.tx_with_same_fee

                            victim_transactions[tx.id].confirmed = True
                            victim_transactions[tx.id].confirmedBlockNumber = blocksCounter

                            number_of_tx_to_be_confirmed -= 1
                            num_confirmed_victim_transactions += 1   

                            ln_txs_to_add_per_fee_level[tx.fee_index] += 1                 

                        if(tx.fee_index in victim_fee_indexes_cache):
                            indexes = victim_fee_indexes_cache[tx.fee_index]
                            for index in indexes:
                                victim_transactions[index].tx_with_same_fee -= delta_txs
                            
                        if(remaining_attacker_txs > 0):
                            # If there are no more attacker transactions to be confirmed, no victim transactions will be added, and we don't care about 
                            # positions of insertion in the mempool

                            if(tx_to_be_added[tx.fee_index] >= delta_txs):
                                tx_to_be_added[tx.fee_index] -= delta_txs
                            else:
                                tx_to_be_added[tx.fee_index] = 0   
                            
                    elif tx.isAttacker:
                        # First, we have the initial number of transactions with the same fee as the attacker's to confirm
                        if(initial_tx_with_same_fee_attacker > 0):
                            
                            if(initial_tx_with_same_fee_attacker > number_of_tx_to_be_confirmed):
                                initial_tx_with_same_fee_attacker -= number_of_tx_to_be_confirmed
                                number_of_tx_to_be_confirmed = 0
                            else:
                                initial_tx_with_same_fee_attacker = 0
                                number_of_tx_to_be_confirmed -= initial_tx_with_same_fee_attacker

                        if(remaining_attacker_txs > number_of_tx_to_be_confirmed):
                            remaining_attacker_txs -= number_of_tx_to_be_confirmed

                            tx_to_be_added[fee_index_attacker] -= number_of_tx_to_be_confirmed

                            new_victim_index = victim_index + number_of_tx_to_be_confirmed
                            number_of_tx_to_be_confirmed = 0

                        else:
                            number_of_tx_to_be_confirmed -= remaining_attacker_txs

                            tx_to_be_added[fee_index_attacker] -= remaining_attacker_txs

                            new_victim_index = victim_index + remaining_attacker_txs
                            remaining_attacker_txs = 0

                        
                        while(victim_index < new_victim_index):
                            victim_transactions[victim_index].submitted = True
                            fee_index = getAverageFeeIndex(tx_count_per_fee_level)
                            victim_transactions[victim_index].fee_index = fee_index
                            victim_transactions[victim_index].tx_with_same_fee = tx_count_per_fee_level[fee_index] + tx_to_be_added[fee_index]
                            victim_transactions[victim_index].attackerTxConfirmedBlockNumber = blocksCounter

                            if not fee_index in victim_fee_indexes_cache:
                                victim_fee_indexes_cache[fee_index] = []

                            victim_fee_indexes_cache[fee_index].append(victim_index)
                            
                            victim_index += 1    

                # end of transactions loop
                for i in range(len(fee_ranges)):
                    tx_to_be_added[i] += ln_txs_to_add_per_fee_level[i]
                

        if(num_confirmed_victim_transactions > 0):
            i = 0

        if(num_confirmed_victim_transactions >= num_channels):
            print("Finished! block ", blocksCounter)

            channels_not_able_to_close = 0
            for tx in victim_transactions:
                if((tx.confirmedBlockNumber - tx.attackerTxConfirmedBlockNumber) > LN_DELAY):
                    channels_not_able_to_close += 1
            print("Channels not able to close in time: ", channels_not_able_to_close)
            return channels_not_able_to_close

        last_total_tx_count = total_tx_count
        last_tx_count_per_fee_level = tx_count_per_fee_level
    
    return None

In [16]:
INITIAL_NUM_ZOMBIE_CHANNELS = 60000
LN_CLOSING_CHANNEL_DELAY = 1000 # ~ 1 week

accumulated_txs = 0 # transactions that would have been already confirmed without the zombie channels transactions

# We read the file containing mempool historical data

i = 1
mempool_data_json_string = ''

while os.path.exists(f"mempool/{i}_mempool"):
    mempool_data_file = open(f"mempool/{i}_mempool", mode = 'r')
    mempool_data_content = mempool_data_file.read()
    
    # We replace call() from file content (it is used for the website to load the JSONP)
    mempool_data_content = mempool_data_content[5 : len(mempool_data_content) - 2]
    
    # I remove the first and the last square brackets, then I will add them again at the end before parsing the JSON,
    # in order to obtain a single merged json of all the mempool data
    mempool_data_content = mempool_data_content[1:]
    mempool_data_content = mempool_data_content[:-1]

    mempool_data_content += ','

    mempool_data_json_string += mempool_data_content
    mempool_data_file.close()

    i += 1

mempool_data_json_string = mempool_data_json_string[:-1]
mempool_data_json_string = f"[{mempool_data_json_string}]"

# Parsing JSON file

mempool_data = json.loads(mempool_data_json_string) 

#simulate(mempool_data, 13, 60000)

y = []
x = []

simulate(mempool_data, 14, 60000)

i = 13
fee_ranges2 = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 17, 20, 25, 30, 40, 50, 60, 70, 80, 100 ]
while(i < len(fee_ranges2)):
    n = simulate(mempool_data, i, 60000)
    y.append(n)
    x.append(fee_ranges2[i])
    print(fee_ranges2[i], "done, n =", n)
    i += 1

fig, ax = plt.subplots()
ax.grid(True)
plt.scatter(x, y)
plt.show() 

fig, ax = plt.subplots()
ax.grid(True)
plt.plot(x, y)
plt.show() 

Finished! block  8279
Channels not able to close in time:  563


563